In [ ]:
import requests
from bs4 import BeautifulSoup 
import pymongo 
from pymongo import MongoClient
import requests
from urllib.parse import urljoin, urlparse, urlunparse, parse_qs, urlencode
import re
import time
import pandas as pd
import sys
from datetime import datetime

# Fights table
##For database menu:
###First layer
Extract the brief information page containing UFC historical fights from 1994 to 2023. 



In [ ]:
mango = pymongo.MongoClient("mongodb://localhost:27017/")
db = mango['422_Group_Project']
collection = db['UFC Completed Fights']

In [ ]:
URL = 'http://www.ufcstats.com/statistics/events/completed?page=all'
#scrap the first 7 pages and save it to file
events = []
links = []
search_page = requests.get(URL, headers = {'User-Agent': 'Mozilla/5.0'}).text 
with open(f'Search Page.htm', 'w') as f:
  f.write(search_page)
# print(f'{f.name} has been saved successfully!')
  soup = BeautifulSoup(search_page,'html.parser') #parse each result page
  results = soup.find_all('tr',{'class':"b-statistics__table-row"}) 

#finding each fight
  for result in results:
    content = result.find('i',{'class':'b-statistics__table-content'})
    place = result.find('td',{'class':'b-statistics__table-col b-statistics__table-col_style_big-top-padding'})
    #each page use first column as spacing, use if function to skip it
    if content:
      link = content.a['href']
      links.append(link)
      text = content.a.text
      title = re.sub("\n[ ]*","",text)
      events.append(title)
      time = content.span.text
      date = re.sub("\n[ ]*","",time)
      local = place.text
      location = re.sub("\n[ ]*","",local)   
      data = {'title': title,
              'date':date,
              'location': location,
              'link':link}
      print(data)
      # collection.insert_one(data)
    else:
      pass
  f.close() 

{'title': 'UFC Fight Night: Yan vs. Dvalishvili', 'date': 'March 11, 2023', 'location': 'Las Vegas, Nevada, USA', 'link': 'http://www.ufcstats.com/event-details/35080a7f406f9ab3'}
{'title': 'UFC 285: Jones vs. Gane', 'date': 'March 04, 2023', 'location': 'Las Vegas, Nevada, USA', 'link': 'http://www.ufcstats.com/event-details/1ccff7f0cfdf85eb'}
{'title': 'UFC Fight Night: Muniz vs. Allen', 'date': 'February 25, 2023', 'location': 'Las Vegas, Nevada, USA', 'link': 'http://www.ufcstats.com/event-details/806975e1b4f47b27'}
{'title': 'UFC Fight Night: Andrade vs. Blanchfield', 'date': 'February 18, 2023', 'location': 'Las Vegas, Nevada, USA', 'link': 'http://www.ufcstats.com/event-details/f21a3d68fb9df387'}
{'title': 'UFC 284: Makhachev vs. Volkanovski', 'date': 'February 11, 2023', 'location': 'Perth, Western Australia, Australia', 'link': 'http://www.ufcstats.com/event-details/01dd4cdc2446f665'}
{'title': 'UFC Fight Night: Lewis vs. Spivac', 'date': 'February 04, 2023', 'location': 'Las 

## Save the link to local HTM files

In [ ]:
for i in range(len(links)):
    url_fight = links[i]
    # Send a GET request to the website
    fight_res = requests.get(url_fight, headers = {'User-Agent': 'Mozilla/5.0'})
    # Create a filename
    filename = "ufc_fight{:03d}.htm".format(i + 1)
    # Save the response to a file
    with open(filename, "w") as f:
        f.write(fight_res.text)
        f.close()
    print("UFC Fight Page Saved Successfully: " + filename)
    # Pause for 5 seconds
    # time.sleep(5)

UFC Fight Page Saved Successfully: ufc_fight001.htm
UFC Fight Page Saved Successfully: ufc_fight002.htm
UFC Fight Page Saved Successfully: ufc_fight003.htm
UFC Fight Page Saved Successfully: ufc_fight004.htm
UFC Fight Page Saved Successfully: ufc_fight005.htm
UFC Fight Page Saved Successfully: ufc_fight006.htm
UFC Fight Page Saved Successfully: ufc_fight007.htm
UFC Fight Page Saved Successfully: ufc_fight008.htm
UFC Fight Page Saved Successfully: ufc_fight009.htm
UFC Fight Page Saved Successfully: ufc_fight010.htm
UFC Fight Page Saved Successfully: ufc_fight011.htm
UFC Fight Page Saved Successfully: ufc_fight012.htm
UFC Fight Page Saved Successfully: ufc_fight013.htm
UFC Fight Page Saved Successfully: ufc_fight014.htm
UFC Fight Page Saved Successfully: ufc_fight015.htm
UFC Fight Page Saved Successfully: ufc_fight016.htm
UFC Fight Page Saved Successfully: ufc_fight017.htm
UFC Fight Page Saved Successfully: ufc_fight018.htm
UFC Fight Page Saved Successfully: ufc_fight019.htm
UFC Fight Pa

## We used 2 methods to insert the fight information to MongoDB:
### 1. for loop to insert_one in each loop 
### 2. create a structured dictionary and then insert_many

#### For each UFC fight, we extract the matches information from that fight night, including win/loss, fighters, winning method, time, etc.
http://www.ufcstats.com/event-details/1ccff7f0cfdf85eb


### Method 1: For each fight match information of each fight night
#### Create second layer of matches information under the main fight menu level

In [ ]:
url = "http://www.ufcstats.com/statistics/events/completed?page=all"
user_agent = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36'}

#save each ufc fight link
page = requests.get(url, headers=user_agent)
soup = BeautifulSoup(page.content, "html.parser")
mainpage = soup.find_all("tr", class_= "b-statistics__table-row")

links = []
for i in mainpage:
    pg = i.find_all("a", class_= "b-link_style_black")
    for l in pg:
        link = l.get("href")
        links.append(link)


In [ ]:
for n in range(len(links)):
    url_fight = links[n]
    # Send a GET request to the website
    fight_res = requests.get(url_fight, headers = user_agent)
    # Create a filename
    filename = "ufc_fight{:03d}.htm".format(n + 1)
    # Save the response to a file
    with open(filename, "w") as f:
        f.write(fight_res.text)
        f.close()
    print("UFC Fight Page Saved Successfully: " + filename)
    # Pause for 5 seconds
    time.sleep(5)

In [ ]:
#soup
t1 = []
for i in range (3):
    filename = "ufc_fight{:03d}.htm".format(i + 1)
    with open(filename, "r", encoding='utf-8')as f:
        html = f.read()
        soup_match = BeautifulSoup(html, "html.parser")
        t1.append(soup_match)
            

In [ ]:
#Outer Layer: Fight Name, Fight Location
#Inner Layer: "match_information": fighters[]
#fighters[]: list of matches in one Fight

t2 = []
for n in t1:
    title_test = n.find_all("span", class_="b-content__title-highlight")
    t = dict()
    for c in title_test:
        co = c.text.strip()
        t["fight"] = co
    dt = n.find("ul", class_="b-list__box-list")
    date_test = dt.find_all("li")[0].get_text(strip=True).replace("Date:", "")
    loca_test = dt.find_all("li")[1].get_text(strip=True).replace("Location:", "")
    t["date"] = date_test
    t["location"] = loca_test
    #match info for each fighter
    fighters = []
    f_info = n.find_all("tr", class_="b-fight-details__table-row b-fight-details__table-row__hover js-fight-details-click")
    for f in f_info:
        summary = {}
        f1 = f.find_all("p", class_="b-fight-details__table-text")[1].get_text(strip=True)
        f2 = f.find_all("p", class_="b-fight-details__table-text")[2].get_text(strip=True)
        summary["Fighter 1"] = f1
        summary["Fighter 2"] = f2
        summary["Winner"] = f1
        
        #match info summary
        wc = f.find_all("td", class_="b-fight-details__table-col l-page_align_left")[1].get_text(strip=True)
        summary["Weight Class"] = wc
        method = f.find_all("td", class_="b-fight-details__table-col l-page_align_left")[2]
        win_method = method.find_all("p")[0].get_text(strip=True)
        summary["Winning Method"] = win_method
        sub_method = method.find_all("p")[1].get_text(strip=True)
        summary["Submission Method"] = sub_method
        
        kd = f.find_all("td", class_="b-fight-details__table-col")[2]
        strike = f.find_all("td", class_="b-fight-details__table-col")[3]
        td = f.find_all("td", class_="b-fight-details__table-col")[4]
        sub = f.find_all("td", class_="b-fight-details__table-col")[5]
        
        stats1 = {}
        kd1 = kd.find_all("p", class_="b-fight-details__table-text")[0].get_text(strip=True)
        str1 = strike.find_all("p", class_="b-fight-details__table-text")[0].get_text(strip=True)
        td1 = td.find_all("p", class_="b-fight-details__table-text")[0].get_text(strip=True)
        sub1 = sub.find_all("p", class_="b-fight-details__table-text")[0].get_text(strip=True)
        stats1["KnockDowns"] = kd1
        stats1["Sig.Strikes"] = str1
        stats1["TakeDowns"] = td1
        stats1["Submission"] = sub1
        summary["Fighter 1 Statistics"] = stats1
        
        stats2 = {}
        kd2 = kd.find_all("p", class_="b-fight-details__table-text")[1].get_text(strip=True)
        str2 = strike.find_all("p", class_="b-fight-details__table-text")[1].get_text(strip=True)
        td2 = td.find_all("p", class_="b-fight-details__table-text")[1].get_text(strip=True)
        sub2 = sub.find_all("p", class_="b-fight-details__table-text")[1].get_text(strip=True)
        stats2["KnockDowns"] = kd2
        stats2["Sig.Strikes"] = str2
        stats2["TakeDowns"] = td2
        stats2["Submission"] = sub2
        summary["Fighter 2 Statistics"] = stats2

        fighters.append(summary)
        t["match information"] = fighters
    
    t2.append(t)

In [ ]:
# establish a connection to MongoDB
mongo = pymongo.MongoClient("mongodb://localhost:27017/")

# print the list of database names in the MongoDB server
print(mongo.list_database_names())

In [ ]:
#establish connection to MongoDB
mongo = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = mongo['ufc']
mycol = mydb['fight1']

mydb.fight1.insert_many(t2)

# print the list of database names in the MongoDB server
print(mongo.list_database_names())


### Method 2: 

In [ ]:
battle = []
for i in range(len(links)):
    filename = "ufc_fight{:03d}.htm".format(i + 1)
    with open(filename, "r", encoding='utf-8')as f:
        html = f.read()
    soup = BeautifulSoup(html, "html.parser")

    name = soup.find("span", class_="b-content__title-highlight").text
    title = re.sub("\n[ ]*","",name)
    #match info for each fighter
    # fighters = []
    info = soup.find_all("tr", class_="b-fight-details__table-row b-fight-details__table-row__hover js-fight-details-click")
    # print(len(info))
    
    for f in info:
        f1 = f.find_all("p", class_="b-fight-details__table-text")[1].get_text(strip=True)
        f2 = f.find_all("p", class_="b-fight-details__table-text")[2].get_text(strip=True)
        
      #match info summary
        wc = f.find_all("td", class_="b-fight-details__table-col l-page_align_left")[1].get_text(strip=True)
        method = f.find_all("td", class_="b-fight-details__table-col l-page_align_left")[2]
        win_method = method.find_all("p")[0].get_text(strip=True)
        sub_method = method.find_all("p")[1].get_text(strip=True)

      
        kd = f.find_all("td", class_="b-fight-details__table-col")[2]
        strike = f.find_all("td", class_="b-fight-details__table-col")[3]
        td = f.find_all("td", class_="b-fight-details__table-col")[4]
        sub = f.find_all("td", class_="b-fight-details__table-col")[5]
      
        #stats1
        kd1 = kd.find_all("p", class_="b-fight-details__table-text")[0].get_text(strip=True)
        str1 = strike.find_all("p", class_="b-fight-details__table-text")[0].get_text(strip=True)
        td1 = td.find_all("p", class_="b-fight-details__table-text")[0].get_text(strip=True)
        sub1 = sub.find_all("p", class_="b-fight-details__table-text")[0].get_text(strip=True)
      
        #stats2
        kd2 = kd.find_all("p", class_="b-fight-details__table-text")[1].get_text(strip=True)
        str2 = strike.find_all("p", class_="b-fight-details__table-text")[1].get_text(strip=True)
        td2 = td.find_all("p", class_="b-fight-details__table-text")[1].get_text(strip=True)
        sub2 = sub.find_all("p", class_="b-fight-details__table-text")[1].get_text(strip=True)


        battle.append({'Win': f1, 'Winning_Stats':{'KnockDowns':kd1,"Sig.Strikes":str1, "TakeDowns":td1,"Submission":sub1},'Lose': f2, 'Losing_Stats':{'KnockDowns':kd2,"Sig.Strikes":str2, "TakeDowns":td2,"Submission":sub2},"Weight Class":wc,"Winning Method":win_method,"Submission Method":sub_method})
#         print(battle)
        
    for d in range(len(battle)):
        collection.update_one({'title':title},{'$set':{f'match{d+1}':battle[d]}})
    print(f'{title} has been imported successfully!')
    battle.clear()


# Fighters table

In [ ]:
# scrape the headers
url = 'http://www.ufcstats.com/statistics/fighters?char=a&page=all'
page = requests.get(url, headers = {'User-Agent': 'Mozilla/5.0'})
doc = BeautifulSoup(page.content, 'html.parser')

headers = []
for header in doc.find_all('th', class_ = 'b-statistics__table-col'):
  headers.append(header.text.strip())

print(headers)

In [ ]:
# scrape fighters info
fighters = []
alphabet = list(string.ascii_lowercase)
for i in range(len(alphabet)):
    url = f'http://www.ufcstats.com/statistics/fighters?char={alphabet[i]}&page=all'
    page = requests.get(url, headers = {'User-Agent': 'Mozilla/5.0'})
    doc = BeautifulSoup(page.content, 'html.parser')
    
    for fighter_row in doc.find_all('tr', class_ = 'b-statistics__table-row')[1:]:
        fighter = {}
        for j, info in enumerate(fighter_row.find_all('td', class_ = 'b-statistics__table-col')):
            if headers[j] == 'Ht.':
                # adjust the units of height
                height = info.text.strip()
                if height != '--':
                    foot = re.search('\d+', height).group()
                    inch = re.search('(\d+)\"', height).group(1)
                    fighter[headers[j]] = int(foot) * 12 + int(inch)
                else:
                    fighter[headers[j]] = None
            elif headers[j] == 'Reach':
                # adjust the units of reach
                reach = info.text.strip()
                if reach != '--':
                    fighter[headers[j]] = int(re.search('(\d+)\..+', reach).group(1))
                else:
                    fighter[headers[j]] = None
            else:
                fighter[headers[j]] = info.text.strip().replace(' lbs.', '')

            
        fighters.append(fighter)

# remove empty list
fighters = list(filter(None, fighters))

pprint.pprint(fighters)

In [ ]:
# check the length of fighters list
len(fighters)

In [ ]:
# connect to mongodb
mango = pymongo.MongoClient('mongodb://localhost:27017/')
db = mango['422_Group_Project']
collection = db['UFC Fighters']

# insert the data
collection.insert_many(fighters)